CONTRASTIVE LOSS

In [1]:
import warnings
import os
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Lambda
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
import tensorflow.keras.backend as K

In [2]:
#dictionary for storing the numpy objects of pairs of images, lables and pairs of images names
arr_dict_X_Y_names = {}

In [3]:
for sample in ['train','valid','test']:
    with open(f'./cropped_numpys/cropped_{sample}_X.npy', 'rb') as f:
         arr_dict_X_Y_names[f'arr_{sample}_X'] = np.load(f)


arr_train_X = arr_dict_X_Y_names['arr_train_X']
arr_valid_X = arr_dict_X_Y_names['arr_valid_X']
arr_test_X = arr_dict_X_Y_names['arr_test_X']

In [4]:
for sample in ['train','valid','test']:
    with open(f'./cropped_numpys/cropped_{sample}_X_names.npy', 'rb') as f:
        arr_dict_X_Y_names[f'{sample}_X_names'] = np.load(f)

train_X_names = arr_dict_X_Y_names['train_X_names']
valid_X_names = arr_dict_X_Y_names['valid_X_names']
test_X_names = arr_dict_X_Y_names['test_X_names']

In [5]:
for sample in ['train','valid','test']:
    with open(f'./cropped_numpys/cropped_{sample}_Y.npy', 'rb') as f:
        arr_dict_X_Y_names[f'arr_{sample}_Y'] = np.load(f)

arr_train_Y = arr_dict_X_Y_names['arr_train_Y']
arr_valid_Y = arr_dict_X_Y_names['arr_valid_Y']
arr_test_Y = arr_dict_X_Y_names['arr_test_Y']

In [6]:
#function creating pairs
def	make_pairs(images, labels, image_names):

	pairImages = []
	pairLabels = []
	pairImagesNames = []
	uniqueClasses = np.unique(labels)

	dict_idx = {i:np.where(labels == i)[0] for i in uniqueClasses}

	for idxA in range(len(images)):
		currentImage = images[idxA]
		label = labels[idxA]
		currentImage_name = image_names[idxA]

		#positive pair
		idxB = np.random.choice(dict_idx[label])
		posImage = images[idxB]
		posImage_name = image_names[idxB]
		pairImages.append([currentImage, posImage])
		pairImagesNames.append([currentImage_name, posImage_name])
		pairLabels.append([1])

		#negative pair
		negLab = np.random.choice([i for i in dict_idx.keys() if i != label])
		negIdx = np.random.choice(dict_idx[negLab])
		negImage = images[negIdx]
		negImage_name = image_names[negIdx]
		pairImages.append([currentImage, negImage])
		pairImagesNames.append([currentImage_name, negImage_name])
		pairLabels.append([0])

	return (np.array(pairImages),np.array(pairLabels)), np.array(pairImagesNames)

In [7]:
(pairTrain, labelTrain), pairNamesTrain = make_pairs(arr_train_X, arr_train_Y, train_X_names)
(pairValid, labelValid), pairNamesValid = make_pairs(arr_valid_X, arr_valid_Y, valid_X_names)
(pairTest, labelTest), pairNamesTest = make_pairs(arr_test_X, arr_test_Y, test_X_names)

In [8]:
print('Number of training pairs:',len(pairTrain))
print('Number of validation pairs:',len(pairValid))
print('Number of test pairs:',len(pairTest))

Number of training pairs: 12000
Number of validation pairs: 4000
Number of test pairs: 4000


In [9]:
dict_pairs = {'train': {'pair_imgs': pairTrain, 'pair_imgs_names':pairNamesTrain,'labels':labelTrain},
                'valid': {'pair_imgs': pairValid,'pair_imgs_names':pairNamesValid,'labels':labelValid},
                'test': {'pair_imgs': pairTest, 'pair_imgs_names':pairNamesTest,'labels':labelTest}}

In [10]:
#exporting the pairs of cropped images, their names and labels (1/0).
for sampl in dict_pairs.keys():
    for n in dict_pairs[sampl].keys():
       with open(f'./pair_numpys/{sampl}_{n}.npy', 'wb') as f:
            np.save(f, dict_pairs[sampl][n])

NN model building with contrastive loss

In [11]:
def euclidean_distance(vectors):
	(featsA, featsB) = vectors
    
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,
		keepdims=True)

	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [12]:
imgA = Input(shape=(224, 224, 3))
imgB = Input(shape=(224, 224, 3))

In [13]:
def build_siamese_model(inputShape, embeddingDim=48):
	inputs = Input(inputShape)

	x = Conv2D(64, (2, 2),  activation="relu")(inputs)
	x = MaxPooling2D(pool_size=(2, 2))(x)
	x = GlobalAveragePooling2D()(x)
	outputs = Dense(embeddingDim)(x)
	
	model = Model(inputs, outputs)
	
	return model

In [14]:
def contrastive_loss(y, preds, margin=1):

	y = tf.cast(y, preds.dtype)

	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

	return loss

In [15]:
featureExtractor = build_siamese_model((224, 224, 3))

In [16]:
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

In [17]:
distance = Lambda(euclidean_distance)([featsA, featsB])

In [18]:
model = Model(inputs=[imgA, imgB], outputs=distance)

In [19]:
model.compile(loss=contrastive_loss, optimizer="adam", metrics=["accuracy"])

In [20]:
history = model.fit(
	[pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
	validation_data=([pairValid[:, 0], pairValid[:, 1]], labelValid[:]),
	epochs=5, verbose = 1)

Epoch 1/5
375/375 [==============================] - 510s 1s/step - loss: 57.4097 - accuracy: 0.3901 - val_loss: 0.2862 - val_accuracy: 0.2275
Epoch 2/5
375/375 [==============================] - 488s 1s/step - loss: 0.3894 - accuracy: 0.3776 - val_loss: 0.2000 - val_accuracy: 0.2595
Epoch 3/5
375/375 [==============================] - 468s 1s/step - loss: 0.2959 - accuracy: 0.3721 - val_loss: 0.2234 - val_accuracy: 0.2467
Epoch 4/5
375/375 [==============================] - 469s 1s/step - loss: 0.2856 - accuracy: 0.3723 - val_loss: 0.1868 - val_accuracy: 0.2928
Epoch 5/5
375/375 [==============================] - 475s 1s/step - loss: 0.2807 - accuracy: 0.3745 - val_loss: 0.2427 - val_accuracy: 0.2473


Predictions

In [21]:
predictions = model.predict([pairTest[:, 0], pairTest[:, 1]])

125/125 [==============================] - 49s 363ms/step
